## Configurações de calibração para cada câmera

Para a câmera V1:
patternSize = (10,7)
squareSize = 30
imgSize = (2592,1944)

Para a câmera V2:
patternSize = (10,7)
squareSize = 30
imgSize = (3280,2464)

Para Webcam:
patternSize = (10,7)
squareSize = 30
imgSize = (1280,960)

Para as imagens antigas:
patternSize = (11,12)
squareSize = 30
imgSize = (960,720)



### Calibração de câmeras - paramêtros intrísicos

In [1]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

useDeng = False
cameraNumber = 4
useFisheye = True

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


Importe as imagens do dataset e configure a escolha da câmera

In [7]:
images = glob.glob('../../../../images/virtual/10x7/opengl3/*.jpg')
patternSize = (10,7)
squareSize = 10
imgSize = (720,720)

#See how images are taken - debug
print(len(images))


50


Reconstrução do tabueleiro 

In [3]:
def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize
    return X

boardPoints = construct3DPoints(patternSize,squareSize)
worldPoints = []
imagePoints = [] 

Funções para remover distorção de imagem

In [ ]:
def undistortFisheye(imgpath,K,D,DIM,axis,method,scale,corners):
    img = cv2.imread(imgpath)
    '''new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, DIM, np.eye(3), balance=0)

    new_K = K.copy()
    new_K[0,0]=K[0,0]*scale
    new_K[1,1]=K[1,1]*scale'''
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    fx = K[0,0]
    fy = K[1,1]
    cx = K[0,2]
    cy = K[1,2]
    undCorners_norm = cv2.fisheye.undistortPoints(corners, K, D)
    undCorners_norm = undCorners_norm.reshape(-1,2)
    undistCorners = np.zeros_like(undCorners_norm)
    for i, (x, y) in enumerate(undCorners_norm):
        px = x*fx + cx
        py = y*fy + cy
        undistCorners[i,0] = px
        undistCorners[i,1] = py    
    cv2.drawChessboardCorners(undistorted_img, patternSize, undistCorners, _)

    axs[axis].imshow(undistorted_img[:,:,::-1])
    axs[axis].axis('off')
    axs[axis].set_title('undistort '+method)
    #cv2.imwrite('undistort'+method+'.png', undistorted_img)

    return corners,undistCorners

def undistortPinhole(imgpath,K,D,DIM,axis,method,corners):
    img = cv2.imread(imgpath)
    
    new_K, roi = cv2.getOptimalNewCameraMatrix(K, D, DIM, 1, DIM)
    undistorted_img = cv2.undistort(img,K,D, None, new_K)  

    undCorners_norm = cv2.undistortPoints(corners, K, D).reshape(-1,2)
    # remove normalization
    fx = new_K[0,0]
    fy = new_K[1,1]
    cx = new_K[0,2]
    cy = new_K[1,2]
    undistCorners = np.zeros_like(undCorners_norm)
    for i, (x, y) in enumerate(undCorners_norm):
        px = x*fx + cx
        py = y*fy + cy
        undistCorners[i,0] = px
        undistCorners[i,1] = py    
    cv2.drawChessboardCorners(undistorted_img, patternSize, undistCorners, _)
    axs[axis].imshow(undistorted_img[:,:,::-1])
    axs[axis].axis('off')
    axs[axis].set_title('undistorted '+method)   


Detectando corners

In [8]:
counter = 0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
    if ret == True:
        print("Corners found!") #- see if corners are found 
        imagePoints.append(corners)
        worldPoints.append(boardPoints)
        counter+=1

        '''cv2.drawChessboardCorners(img, patternSize, cornersRefined, ret)
        plt.figure(figsize=(4, 3), dpi=200)
        plt.imshow(img[:,:,::-1])
        plt.axis('off')
        plt.title(fname)'''

print("using " + str(counter))

using 0


Calibração de câmera

In [56]:
if useFisheye:
    flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
    calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)
    ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPoints), -2), imagePoints, imgSize, None, None,
                                                                flags=flagsCalib,criteria=calibrateCriteria)
else:
    flagsCalib = cv2.CALIB_RATIONAL_MODEL
    ret, cameraMatrix, k, rvecs, tvecs = cv2.calibrateCamera(worldPoints, imagePoints, imgSize, None, None,
                                                               flags=flagsCalib)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", k)


Using 20 of 20 images
RMS re-projection error: 0.2547153364603549
Camera Matrix:
 [[2.97646232e+03 0.00000000e+00 1.18425363e+03]
 [0.00000000e+00 2.98577128e+03 1.08503468e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:
 [[-0.12923276]
 [ 0.18717814]
 [-1.35831681]
 [ 2.51654783]]


In [53]:
if useFisheye:
    flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
    calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)
    ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPoints), -2), imagePoints, imgSize, None, None,
                                                                flags=flagsCalib,criteria=calibrateCriteria)
else:
    flagsCalib = cv2.CALIB_RATIONAL_MODEL
    ret, cameraMatrix, k, rvecs, tvecs, stdDeviationsIntrinsics, stdDeviationsExtrinsics, perViewErrors = cv2.calibrateCameraExtended(worldPoints, imagePoints, imgSize, None, None, flags=flagsCalib)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", k)
print(len(perViewErrors))


Using 20 of 20 images
RMS re-projection error: 0.2276229064937841
Camera Matrix:
 [[2.91055449e+03 0.00000000e+00 1.22119271e+03]
 [0.00000000e+00 2.91732984e+03 1.06036950e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:
 [[-1.14124763e+01  1.65790903e+02  2.12570623e-03 -3.98145110e-03
   3.15381135e+01 -1.09700574e+01  1.60681780e+02  1.04498115e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
20


In [54]:
#reprojection error: 

mean = np.mean(perViewErrors)
median = np.median(perViewErrors)

print("the mean error is:", mean)
print("the RMS error is:", ret)
print("the median error is:", median)

the mean error is: 0.2209752722527008
the RMS error is: 0.2276229064937841
the median error is: 0.2035584664058449


### Análise dos resultados

In [ ]:
idx = 3
pathImg = images[idx]
img = cv2.imread(pathImg)

if useFisheye:
    fig, axs = plt.subplots(1,3,figsize=(16, 4), dpi=500)
    _, corners = cv2.findChessboardCorners(img, patternSize, None)
    axs[0].imshow(img[:,:,::-1])
    axs[0].axis('off')
    axs[0].set_title('distorted')
    #cv2.imwrite('undistortNOriginal.png', img)
    undistortFisheye(pathImg,cameraMatrix,k,imgSize,1,'openCV',1,corners)
    if useDeng:
        k[0] = 0.2377
        k[1] = -0.3752
        k[2] = 6.6712
        k[3] = -27.4156
    else:
        if cameraNumber == 1:
            k[0] = 0.395621
            k[1] = 0.633705 
            k[2] = -2.41723
            k[3] = 2.11079
            cameraMatrix[0][0] = 720.313
            cameraMatrix[1][1] = 719.521
            cameraMatrix[0][2] = 481.014
            cameraMatrix[1][2] = 360.991
        elif cameraNumber == 2: 
            k[0] = 0.368917
            k[1] = 1.50111
            k[2] = -7.94126
            k[3] = 11.9171
            cameraMatrix[0][0] = 768.113
            cameraMatrix[1][1] = 767.935
            cameraMatrix[0][2] = 472.596
            cameraMatrix[1][2] = 350.978
        elif cameraNumber == 3:
            k[0] = 0.276114
            k[1] = 2.09465
            k[2] = -9.97956
            k[3] = 14.1921
            cameraMatrix[0][0] = 728.237
            cameraMatrix[1][1] = 729.419 
            cameraMatrix[0][2] = 459.854
            cameraMatrix[1][2] = 351.59
        else:
            k[0] = 0.400774
            k[1] = 1.15995
            k[2] = -7.10257
            k[3] = 11.415
            cameraMatrix[0][0] = 750.149
            cameraMatrix[1][1] = 748.903
            cameraMatrix[0][2] = 492.144
            cameraMatrix[1][2] = 350.213

    corners,undistCorners = undistortFisheye(pathImg,cameraMatrix,k,imgSize,2,'MATLAB',1,corners)
else:
    fig, axs = plt.subplots(1,2,figsize=(7, 4), dpi=500)
    _, corners = cv2.findChessboardCorners(img, patternSize, None)
    #cv2.drawChessboardCorners(img, patternSize, corners, _)
    axs[0].imshow(img[:,:,::-1])
    axs[0].axis('off')
    axs[0].set_title('distorted')  
    newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, k, imgSize, 1, imgSize)
    undistorted_img = cv2.undistort(img,cameraMatrix,k, None, newCameraMatrix)  
    cv2.drawChessboardCorners(undistorted_img, patternSize, corners, _)
    axs[1].imshow(undistorted_img[:,:,::-1])
    axs[1].axis('off')
    axs[1].set_title('undistorted')  
    undistortPinhole(pathImg,cameraMatrix,k,imgSize,1,'openCV',corners)


In [ ]:
idx = 8

imgpoints, _  = cv2.fisheye.projectPoints(objectPoints=boardPoints.reshape(-1,1,3),
                        rvec=R[idx].reshape(-1,3),
                        tvec = t[idx].reshape(-1,3),
                        K=cameraMatrix,
                        D=k)

img = cv2.imread(images[idx])
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, cornerspoints = cv2.findChessboardCornersSB(gray, patternSize, None)

imgpoints = imgpoints.reshape(-1,2).astype(float)

cornerspoints = cornerspoints.reshape(-1,2)

residual = cornerspoints-imgpoints
plt.scatter(residual[:,0],residual[:,1])
plt.title('Residual plot XY image: {0}'.format(images[idx]))
plt.grid('on')
